# Importing libraries and defining path and filenames

In this block of code, the path and name of the files is defined, and the libraries that are going to be used are imported.

In [1]:
import pandas as pd
import pyodbc as pdb
from datetime import datetime
import os

mainpath ='/home/ale/Downloads'
f1 = '2010-02.csv'
f2 = '2021-02.csv'

# Connection to database

Here, the connection to the database is created.

In [2]:
server = 'localhost' 
database = 'TestDB' 
username = 'ale' 
password = 'my-pa55w0rd' 
cnxn = pdb.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password,autocommit=True)
cursor = cnxn.cursor()

# Table creation

The table where the data will be stored is created in the following code.

If the table already exists, it's dropped, and the it's created.

In [3]:
sql = "DROP TABLE IF EXISTS Viajes"
cursor.execute(sql)
sql = "CREATE TABLE Viajes(ID INT PRIMARY KEY, ID_bici INT, fecha DATE, duracion TIME, est_salida INT, est_llegada INT)"
cursor.execute(sql)
print('The table hass been created succesfully!')

The table hass been created succesfully!


# First transformations

From the first file, a dataframe is created, giving the data the exact format for the table to receive it.

In [4]:
fullpath = os.path.join(mainpath,f1)
data = pd.read_csv(fullpath)

data['Hora_Retiro'] = data['Hora_Retiro'].str.slice(0, 8)
data['Hora_Arribo'] = data['Hora_Arribo'].str.slice(0, 8)

df1 = pd.DataFrame(data)
duracion = pd.to_datetime(df1.Hora_Arribo) - pd.to_datetime(df1.Hora_Retiro)

df1['duracion'] = duracion
df1 = df1.drop(['Genero_Usuario', 'Edad_Usuario', 'Hora_Retiro', 'Fecha_Arribo', 'Hora_Arribo'], axis=1)
df1.columns = ['ID_bici', 'est_salida', 'fecha', 'est_llegada', 'duracion']
df1['ID'] = df1.index
df1 = df1[['ID','ID_bici', 'fecha', 'duracion', 'est_salida', 'est_llegada']]
df1['duracion'] = df1['duracion'].astype(str).str[-18:-10]
df1.head()


,ID,ID_bici,fecha,duracion,est_salida,est_llegada
0,0,69,2010-02-16,00:03:05,85,85
1,1,11,2010-02-16,00:28:54,85,26
2,2,43,2010-02-16,00:28:01,85,13
3,3,826,2010-02-16,00:16:33,22,85
4,4,662,2010-02-16,00:58:02,27,74


# First insertion

In this block of code, the first data set is inserted into the table.

In [5]:
for index, row in df1.iterrows():
    cursor.execute("INSERT INTO Viajes (ID,ID_bici,fecha,duracion,est_salida,est_llegada) values(?,?,?,?,?,?)", row.ID, row.ID_bici, row.fecha, row.duracion, row.est_salida, row.est_llegada)

# Second tranformations

Here, the second file goes through the same transformations that the first one got, so that everything gets the same format.

In [6]:
fullpath = os.path.join(mainpath,f2)
data = pd.read_csv(fullpath)

data['Hora_Retiro'] = data['Hora_Retiro'].str.slice(0, 8)
data['Hora_Arribo'] = data['Hora_Arribo'].str.slice(0, 8)

df2 = pd.DataFrame(data)
duracion2 = pd.to_datetime(df2.Hora_Arribo) - pd.to_datetime(df2.Hora_Retiro)

df2['duracion'] = duracion2
df2 = df2.drop(['Genero_Usuario', 'Edad_Usuario', 'Hora_Retiro', 'Fecha_Arribo', 'Hora_Arribo'], axis=1)
df2.columns = ['ID_bici', 'est_salida', 'fecha', 'est_llegada', 'duracion']
df2['ID'] = df2.index
df2 = df2[['ID','ID_bici', 'fecha', 'duracion', 'est_salida', 'est_llegada']]
df2['duracion'] = df2['duracion'].astype(str).str[-18:-10]
df2['fecha'] = pd.to_datetime(df2['fecha'], format='%d/%m/%Y')
df2.head()

,ID,ID_bici,fecha,duracion,est_salida,est_llegada
0,0,10427.0,2021-02-01,00:07:14,390.0,315
1,1,11429.0,2021-02-01,00:06:07,254.0,272
2,2,9959.0,2021-02-01,00:11:01,476.0,24
3,3,4044.0,2021-02-01,00:14:38,256.0,147
4,4,9986.0,2021-02-01,00:28:24,450.0,105


# Step before update

Since the primary key of the table is the index of the dataframe, the new ID has to start from the last ID inserted. Therefore, this block queries the max ID and overwrites the ID column of the dataframe from that number up.

Also, two of the columns have float data type, we have to change that in order to have the same format as the destiny table.

Something else to consider is that the second file has NULL values, since those rows that have NULL values in any column, have all columns as NULL, the treatment given is to just drop those rows.

In [7]:
sql = "SELECT MAX(ID) FROM Viajes"
cursor.execute(sql)
last_ID = cursor.fetchall()

total_rows = df2.shape[0]

n_first_ID = last_ID[0][0] + 1
n_last_ID = n_first_ID + total_rows

print(n_first_ID, n_last_ID)

n_ID= list(range(n_first_ID, n_last_ID))

df2['n_ID'] = n_ID

df2 = df2.drop(['ID'], axis=1)
df2 = df2[['n_ID','ID_bici', 'fecha', 'duracion', 'est_salida', 'est_llegada']]
df2.columns = ['ID','ID_bici', 'fecha','duracion','est_salida', 'est_llegada']

df3 = df2.dropna(axis = 0, how ='any')
#display(df2.dtypes)


df3['est_salida'] = df3['est_salida'].astype(int)
df3['ID_bici'] = df3['ID_bici'].astype(int)
df3.head()

1057 290842


/home/ale/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ale/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ID,ID_bici,fecha,duracion,est_salida,est_llegada
0,1057,10427,2021-02-01,00:07:14,390,315
1,1058,11429,2021-02-01,00:06:07,254,272
2,1059,9959,2021-02-01,00:11:01,476,24
3,1060,4044,2021-02-01,00:14:38,256,147
4,1061,9986,2021-02-01,00:28:24,450,105


# Update

Here, we simply insert the new values in the table.

In [8]:
for index, row in df3.iterrows():
    cursor.execute("INSERT INTO Viajes (ID,ID_bici,fecha,duracion,est_salida,est_llegada) values(?,?,?,?,?,?)", row.ID, row.ID_bici, row.fecha, row.duracion, row.est_salida, row.est_llegada)

# Select

Here we do a simple SELECT query to check our results.
Since they are a lot, we'll just do a count.

In [ ]:
sql = "SELECT * FROM Viajes"
cursor.execute(sql)
result = cursor.fetchall()
for i in result:
    print(i)

(0, 69, datetime.date(2010, 2, 16), datetime.time(0, 3, 5), 85, 85)
(1, 11, datetime.date(2010, 2, 16), datetime.time(0, 28, 54), 85, 26)
(2, 43, datetime.date(2010, 2, 16), datetime.time(0, 28, 1), 85, 13)
(3, 826, datetime.date(2010, 2, 16), datetime.time(0, 16, 33), 22, 85)
(4, 662, datetime.date(2010, 2, 16), datetime.time(0, 58, 2), 27, 74)
(5, 8, datetime.date(2010, 2, 16), datetime.time(0, 17, 33), 85, 20)
(6, 5, datetime.date(2010, 2, 16), datetime.time(0, 16, 3), 85, 20)
(7, 680, datetime.date(2010, 2, 16), datetime.time(0, 16, 38), 20, 25)
(8, 618, datetime.date(2010, 2, 16), datetime.time(0, 18, 10), 20, 17)
(9, 812, datetime.date(2010, 2, 16), datetime.time(0, 6), 49, 72)
(10, 711, datetime.date(2010, 2, 16), datetime.time(0, 7, 27), 66, 49)
(11, 651, datetime.date(2010, 2, 16), datetime.time(0, 7, 24), 15, 25)
(12, 578, datetime.date(2010, 2, 16), datetime.time(0, 6, 27), 25, 6)
(13, 578, datetime.date(2010, 2, 16), datetime.time(0, 8, 48), 6, 8)
(14, 615, datetime.date(20

(1490, 7316, datetime.date(2021, 2, 1), datetime.time(0, 6, 43), 183, 35)
(1491, 6859, datetime.date(2021, 2, 1), datetime.time(0, 13, 35), 31, 96)
(1492, 7411, datetime.date(2021, 2, 1), datetime.time(1, 40, 55), 79, 136)
(1493, 12411, datetime.date(2021, 2, 1), datetime.time(0, 28, 2), 392, 390)
(1494, 12215, datetime.date(2021, 2, 1), datetime.time(0, 14, 19), 431, 338)
(1495, 15116, datetime.date(2021, 2, 1), datetime.time(0, 26, 49), 260, 247)
(1496, 11091, datetime.date(2021, 2, 1), datetime.time(0, 9, 18), 141, 3)
(1497, 7013, datetime.date(2021, 2, 1), datetime.time(0, 11, 24), 433, 374)
(1498, 12094, datetime.date(2021, 2, 1), datetime.time(0, 4, 28), 86, 35)
(1499, 10909, datetime.date(2021, 2, 1), datetime.time(0, 7, 52), 6, 9)
(1500, 8315, datetime.date(2021, 2, 1), datetime.time(0, 22, 6), 177, 211)
(1501, 12002, datetime.date(2021, 2, 1), datetime.time(0, 16, 48), 301, 356)
(1502, 6999, datetime.date(2021, 2, 1), datetime.time(0, 25, 25), 358, 75)
(1503, 12377, datetime.d

(2826, 12327, datetime.date(2021, 2, 1), datetime.time(0, 3, 3), 182, 164)
(2827, 12451, datetime.date(2021, 2, 1), datetime.time(0, 9, 38), 8, 32)
(2828, 10781, datetime.date(2021, 2, 1), datetime.time(0, 18, 38), 134, 59)
(2829, 9545, datetime.date(2021, 2, 1), datetime.time(0, 10, 10), 103, 52)
(2830, 10854, datetime.date(2021, 2, 1), datetime.time(0, 3, 46), 4, 24)
(2831, 4304, datetime.date(2021, 2, 1), datetime.time(0, 27, 13), 447, 187)
(2832, 11507, datetime.date(2021, 2, 1), datetime.time(0, 7, 9), 343, 345)
(2833, 11413, datetime.date(2021, 2, 1), datetime.time(0, 4, 28), 125, 3)
(2834, 8950, datetime.date(2021, 2, 1), datetime.time(0, 5, 10), 18, 85)
(2835, 7785, datetime.date(2021, 2, 1), datetime.time(0, 8, 55), 70, 84)
(2836, 10631, datetime.date(2021, 2, 1), datetime.time(0, 29, 5), 212, 72)
(2837, 9243, datetime.date(2021, 2, 1), datetime.time(0, 24, 34), 21, 74)
(2838, 8992, datetime.date(2021, 2, 1), datetime.time(0, 21), 279, 349)
(2839, 10517, datetime.date(2021, 2,

(4212, 7686, datetime.date(2021, 2, 1), datetime.time(0, 11, 53), 148, 148)
(4213, 9145, datetime.date(2021, 2, 1), datetime.time(0, 19, 37), 2, 48)
(4214, 9404, datetime.date(2021, 2, 1), datetime.time(0, 17, 57), 24, 166)
(4215, 12519, datetime.date(2021, 2, 1), datetime.time(0, 43, 34), 14, 27)
(4216, 7573, datetime.date(2021, 2, 1), datetime.time(0, 13, 39), 179, 182)
(4217, 8526, datetime.date(2021, 2, 1), datetime.time(0, 7, 32), 10, 254)
(4218, 7464, datetime.date(2021, 2, 1), datetime.time(0, 50, 57), 255, 140)
(4219, 10646, datetime.date(2021, 1, 31), datetime.time(19, 43, 27), 280, 280)
(4220, 7709, datetime.date(2021, 2, 1), datetime.time(0, 3, 59), 180, 63)
(4221, 9119, datetime.date(2021, 2, 1), datetime.time(0, 40, 23), 369, 440)
(4222, 8473, datetime.date(2021, 2, 1), datetime.time(0, 30, 50), 71, 251)
(4223, 2488, datetime.date(2021, 2, 1), datetime.time(0, 24, 7), 1, 238)
(4224, 8090, datetime.date(2021, 2, 1), datetime.time(0, 28, 7), 1, 287)
(4225, 6865, datetime.dat

(5596, 7590, datetime.date(2021, 2, 1), datetime.time(0, 3, 10), 164, 164)
(5597, 8873, datetime.date(2021, 2, 1), datetime.time(0, 10, 53), 255, 128)
(5598, 8073, datetime.date(2021, 2, 1), datetime.time(0, 29, 24), 284, 326)
(5599, 10024, datetime.date(2021, 2, 1), datetime.time(0, 6, 38), 394, 426)
(5600, 8810, datetime.date(2021, 2, 1), datetime.time(0, 16, 13), 396, 433)
(5601, 8036, datetime.date(2021, 2, 1), datetime.time(0, 26, 30), 24, 98)
(5602, 9540, datetime.date(2021, 2, 1), datetime.time(0, 3, 57), 58, 49)
(5603, 6976, datetime.date(2021, 2, 1), datetime.time(0, 23, 31), 172, 172)
(5604, 15254, datetime.date(2021, 2, 1), datetime.time(0, 37, 36), 33, 33)
(5605, 10825, datetime.date(2021, 2, 1), datetime.time(0, 33, 36), 70, 76)
(5606, 11300, datetime.date(2021, 2, 1), datetime.time(0, 18, 58), 333, 348)
(5607, 8922, datetime.date(2021, 2, 1), datetime.time(0, 8, 54), 172, 49)
(5608, 6790, datetime.date(2021, 2, 1), datetime.time(0, 9, 4), 193, 194)
(5609, 10377, datetime.

...

(12273, 12124, datetime.date(2021, 2, 2), datetime.time(0, 6, 2), 125, 3)
(12274, 12469, datetime.date(2021, 2, 2), datetime.time(0, 21, 3), 276, 1)
(12275, 4238, datetime.date(2021, 2, 2), datetime.time(0, 17, 2), 282, 148)
(12276, 12379, datetime.date(2021, 2, 2), datetime.time(0, 29, 25), 124, 218)
(12277, 11682, datetime.date(2021, 2, 2), datetime.time(0, 10, 38), 240, 210)
(12278, 7858, datetime.date(2021, 2, 2), datetime.time(0, 4, 55), 26, 48)
(12279, 9081, datetime.date(2021, 2, 2), datetime.time(0, 4, 52), 17, 1)
(12280, 7606, datetime.date(2021, 2, 2), datetime.time(0, 8, 23), 61, 138)
(12281, 11584, datetime.date(2021, 2, 2), datetime.time(0, 15, 41), 133, 182)
(12282, 6940, datetime.date(2021, 2, 2), datetime.time(0, 9, 47), 158, 132)
(12283, 10204, datetime.date(2021, 2, 2), datetime.time(0, 13, 43), 14, 36)
(12284, 12211, datetime.date(2021, 2, 2), datetime.time(0, 22, 37), 174, 109)
(12285, 11829, datetime.date(2021, 2, 2), datetime.time(0, 25, 18), 75, 151)
(12286, 1140

(13575, 11859, datetime.date(2021, 2, 2), datetime.time(0, 4, 12), 51, 44)
(13576, 8456, datetime.date(2021, 2, 2), datetime.time(0, 32), 271, 74)
(13577, 11134, datetime.date(2021, 2, 2), datetime.time(0, 6, 11), 99, 10)
(13578, 10053, datetime.date(2021, 2, 2), datetime.time(0, 7, 9), 146, 179)
(13579, 10061, datetime.date(2021, 2, 2), datetime.time(0, 8, 6), 148, 153)
(13580, 8630, datetime.date(2021, 2, 2), datetime.time(0, 14, 33), 63, 249)
(13581, 9759, datetime.date(2021, 2, 2), datetime.time(0, 7, 48), 59, 34)
(13582, 12259, datetime.date(2021, 2, 2), datetime.time(0, 12, 41), 271, 96)
(13583, 11280, datetime.date(2021, 2, 2), datetime.time(0, 13, 48), 42, 174)
(13584, 7449, datetime.date(2021, 2, 2), datetime.time(0, 21), 197, 25)
(13585, 7764, datetime.date(2021, 2, 2), datetime.time(0, 31, 1), 97, 1)
(13586, 10117, datetime.date(2021, 2, 2), datetime.time(0, 18, 45), 217, 146)
(13587, 4023, datetime.date(2021, 2, 2), datetime.time(0, 15, 23), 333, 326)
(13588, 7649, datetime

(14967, 9226, datetime.date(2021, 2, 2), datetime.time(0, 19, 34), 23, 16)
(14968, 8240, datetime.date(2021, 2, 2), datetime.time(0, 17, 47), 93, 48)
(14969, 11037, datetime.date(2021, 2, 2), datetime.time(1, 12, 19), 321, 321)
(14970, 8343, datetime.date(2021, 2, 2), datetime.time(0, 23, 31), 167, 37)
(14971, 15229, datetime.date(2021, 2, 2), datetime.time(0, 23, 20), 327, 391)
(14972, 8806, datetime.date(2021, 2, 2), datetime.time(0, 24, 11), 346, 8)
(14973, 4243, datetime.date(2021, 2, 2), datetime.time(0, 7, 58), 88, 104)
(14974, 7320, datetime.date(2021, 2, 2), datetime.time(0, 25, 17), 110, 340)
(14975, 12420, datetime.date(2021, 2, 2), datetime.time(0, 6, 17), 143, 142)
(14976, 7359, datetime.date(2021, 2, 2), datetime.time(0, 29, 35), 394, 167)
(14977, 8003, datetime.date(2021, 2, 2), datetime.time(0, 24, 7), 60, 196)
(14978, 8821, datetime.date(2021, 2, 2), datetime.time(0, 4, 36), 73, 84)
(14979, 10767, datetime.date(2021, 2, 2), datetime.time(0, 17, 51), 58, 318)
(14980, 112

(16455, 12095, datetime.date(2021, 2, 3), datetime.time(0, 12, 18), 480, 240)
(16456, 9748, datetime.date(2021, 2, 3), datetime.time(0, 6, 30), 48, 88)
(16457, 12077, datetime.date(2021, 2, 3), datetime.time(0, 6, 45), 157, 158)
(16458, 10844, datetime.date(2021, 2, 3), datetime.time(0, 21, 39), 50, 88)
(16459, 7260, datetime.date(2021, 2, 3), datetime.time(0, 28, 26), 280, 280)
(16460, 6778, datetime.date(2021, 2, 3), datetime.time(0, 15, 9), 259, 265)
(16461, 1731, datetime.date(2021, 2, 3), datetime.time(0, 35, 28), 176, 197)
(16462, 10015, datetime.date(2021, 2, 3), datetime.time(0, 5, 43), 184, 45)
(16463, 15182, datetime.date(2021, 2, 3), datetime.time(0, 27, 30), 296, 257)
(16464, 7567, datetime.date(2021, 2, 3), datetime.time(0, 4, 42), 45, 69)
(16465, 11033, datetime.date(2021, 2, 3), datetime.time(0, 13, 41), 372, 386)
(16466, 15176, datetime.date(2021, 2, 3), datetime.time(0, 45, 21), 391, 391)
(16467, 4236, datetime.date(2021, 2, 3), datetime.time(0, 10, 48), 45, 149)
(1646

(17928, 8101, datetime.date(2021, 2, 2), datetime.time(0, 9, 46), 56, 77)
(17929, 11693, datetime.date(2021, 2, 2), datetime.time(0, 3, 28), 82, 87)
(17930, 4072, datetime.date(2021, 2, 2), datetime.time(0, 3, 36), 142, 59)
(17931, 9212, datetime.date(2021, 2, 2), datetime.time(0, 26, 8), 70, 351)
(17932, 15189, datetime.date(2021, 2, 2), datetime.time(0, 12, 20), 160, 33)
(17933, 9166, datetime.date(2021, 2, 2), datetime.time(0, 19, 55), 29, 36)
(17934, 8578, datetime.date(2021, 2, 2), datetime.time(0, 19, 38), 47, 480)
(17935, 9178, datetime.date(2021, 2, 2), datetime.time(0, 24, 31), 96, 279)
(17936, 12377, datetime.date(2021, 2, 2), datetime.time(0, 36, 48), 126, 136)
(17937, 9959, datetime.date(2021, 2, 2), datetime.time(0, 4, 47), 136, 126)
(17938, 9157, datetime.date(2021, 2, 2), datetime.time(0, 3, 1), 44, 44)
(17939, 7721, datetime.date(2021, 2, 2), datetime.time(0, 18, 33), 251, 189)
(17940, 9194, datetime.date(2021, 2, 2), datetime.time(0, 6, 49), 14, 60)
(17941, 8122, datet